#Imports de Bibliotecas - Spark



In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/nulll

!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

!pip install -q findspark
!pip install -q pyspark
!pip install py4j

import os
import sys

import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,517 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,817 kB]
Fetched 4,718 kB in 2s (2,465 kB/s)
Reading package lists... Done
Building dependency tr

#Import the Saúde and Trabalho XLSX files from IBGE public page

In [ ]:
#Import COVID-19 dataset from IBGE public website
import requests

#df = spark.read.csv('/content/' + csv+file_name, header=True, sep=",", inferSchema=True)

In [ ]:
#Healthy IBGE dataset file - November.2020 -> Saúde
path = "https://ftp.ibge.gov.br/Trabalho_e_Rendimento/Pesquisa_Nacional_por_Amostra_de_Domicilios_PNAD_COVID19/Mensal/Tabelas/pnad_covid19_202011_saude_BR_GR_UF.xlsx"
req = requests.get(path)
url_content = req.content

csv_file_name = 'covid_202011_saude_UF.xlsx'
csv_file = open(csv_file_name, 'wb')

csv_file.write(url_content)
csv_file.close()

In [ ]:
#Work IBGE dataset file - November.2020 -> Trabalho
path = "https://ftp.ibge.gov.br/Trabalho_e_Rendimento/Pesquisa_Nacional_por_Amostra_de_Domicilios_PNAD_COVID19/Mensal/Tabelas/pnad_covid19_202011_trabalho_BR_GR_UF.xlsx"
req = requests.get(path)
url_content = req.content

csv_file_name = 'covid_202011_trabalho_UF.xlsx'
csv_file = open(csv_file_name, 'wb')

csv_file.write(url_content)
csv_file.close()

In [ ]:
import pandas as pd

In [ ]:
#Read Healthy/Saude XLSX file - Dados com detalhamento por UF

# File path to the Excel file
file_path = 'covid_202011_saude_UF.xlsx'

# Read the Excel sheet, loading rows 8 and 9 as headers
headers = pd.read_excel(file_path, sheet_name="Saúde UF", skiprows=7, nrows=2, header=None)

#Update all values in the second row to NaN (None)
headers.iloc[1] = None

# Merge the two header rows into one
merged_headers = headers.apply(lambda x: ' '.join(x.dropna().astype(str)), axis=0)

# Read the rest of the data, skipping the merged header rows
df_saude = pd.read_excel(file_path, sheet_name="Saúde UF", skiprows=9, header=None)

# Assign the merged headers as the column names
df_saude.columns = merged_headers

In [ ]:
#Read Healthy/Saude XLSX file - dados somente nivel Brasil, sem detalhamento por UF

# File path to the Excel file
file_path = 'covid_202011_saude_UF.xlsx'

# Read the Excel sheet, loading rows 8 and 9 as headers
headers = pd.read_excel(file_path, sheet_name="Saúde BR e GR", skiprows=8, nrows=2, header=None)

#Update all values in the second row to NaN (None)
headers.iloc[1] = None

# Merge the two header rows into one
merged_headers = headers.apply(lambda x: ' '.join(x.dropna().astype(str)), axis=0)

# Read the rest of the data, skipping the merged header rows
df_saude_br = pd.read_excel(file_path, sheet_name="Saúde BR e GR", skiprows=10, header=None)

# Assign the merged headers as the column names
df_saude_br.columns = merged_headers

In [ ]:
# Remove all columns that start with "Situação"
df_saude = df_saude.loc[:, ~df_saude.columns.str.startswith("Situação")]
# Display the DataFrame
df_saude.head()

,Indicador,Nível Territorial,Abertura Territorial,Variável de abertura 1,Categoria de abertura 1,Variável de abertura 2,Categoria de abertura 2,Maio,Junho,Julho,Agosto,Setembro,Outubro,Novembro
0,Pessoas que apresentaram algum dos sintoma(s) ...,Unidade da Federação,Rondônia,-,-,-,-,181.943330,134.270923,91.120006,96.641585,77.548385,51.144917,41.357501
1,CV - Pessoas que apresentaram algum dos sintom...,Unidade da Federação,Rondônia,-,-,-,-,8.318182,7.469208,8.937854,10.752584,11.760041,14.545605,15.459794
2,Percentual de pessoas que apresentaram algum d...,Unidade da Federação,Rondônia,-,-,-,-,10.203347,7.523230,5.100973,5.405325,4.333607,2.855605,2.307125
3,CV - Percentual de pessoas que apresentaram al...,Unidade da Federação,Rondônia,-,-,-,-,8.318182,7.469208,8.937854,10.752584,11.760041,14.545605,15.459794
4,Pessoas que apresentaram sintomas referenciado...,Unidade da Federação,Rondônia,-,-,-,-,29.952155,27.137167,15.521570,24.030210,11.512330,9.253908,7.788923


In [ ]:
# Remove all columns that start with "Situação" - dados ao nível Brasil
df_saude_br = df_saude_br.loc[:, ~df_saude_br.columns.str.startswith("Situação")]
# Display the DataFrame
df_saude_br.head()

,Indicador,Nível Territorial,Abertura Territorial,Variável de abertura 1,Categoria de abertura 1,Variável de abertura 2,Categoria de abertura 2,Maio,Junho,Julho,Agosto,Setembro,Outubro,Novembro
0,Pessoas que apresentaram algum dos sintoma(s) ...,País,Brasil,Sexo,Total,-,-,24012.308055,15506.473518,13793.338769,12135.650022,9237.127768,7810.718422,7956.195690
1,Pessoas que apresentaram algum dos sintoma(s) ...,País,Brasil,Sexo,Homem,-,-,10406.815129,6685.178015,5888.352772,5198.160462,3995.949215,3328.752283,3430.951501
2,Pessoas que apresentaram algum dos sintoma(s) ...,País,Brasil,Sexo,Mulher,-,-,13605.492926,8821.295504,7904.985997,6937.489559,5241.178553,4481.966139,4525.244189
3,Pessoas que apresentaram algum dos sintoma(s) ...,País,Brasil,Grupos de Idade,Total,-,-,24012.308055,15506.473518,13793.338769,12135.650022,9237.127768,7810.718422,7956.195690
4,Pessoas que apresentaram algum dos sintoma(s) ...,País,Brasil,Grupos de Idade,0 a 9 anos,-,-,1735.704555,1194.309017,1076.652310,1208.191614,840.352628,815.966250,868.646497


In [ ]:
#Read Work/Trabalho XLSX file

# File path to the Excel file
file_path = 'covid_202011_trabalho_UF.xlsx'

# Read the Excel sheet, loading rows 8 and 9 as headers
headers = pd.read_excel(file_path, sheet_name="Trabalho UF", skiprows=5, nrows=2, header=None)

#Update all values in the second row to NaN (None)
headers.iloc[1] = None

# Merge the two header rows into one
merged_headers = headers.apply(lambda x: ' '.join(x.dropna().astype(str)), axis=0)

# Read the rest of the data, skipping the merged header rows
df_trabalho = pd.read_excel(file_path, sheet_name="Trabalho UF", skiprows=9, header=None)

# Assign the merged headers as the column names
df_trabalho.columns = merged_headers

# Remove all columns that start with "Situação"
df_trabalho = df_trabalho.loc[:, ~df_trabalho.columns.str.startswith("Situação")]

# Display the DataFrame
df_trabalho.head()

,Indicador,Nível Territorial,Abertura Territorial,Variável de abertura 1,Categoria de abertura 1,Variável de abertura 2,Categoria de abertura 2,Maio,Junho,Julho,Agosto,Setembro,Outubro,Novembro
0,Pessoas de 14 anos ou mais de idade (mil pessoas),Unidade da Federação,Rondônia,-,-,-,-,1396.530675,1399.168148,1402.187864,1402.010782,1403.457753,1406.189257,1412.303094
1,CV - Pessoas de 14 anos ou mais de idade (%),Unidade da Federação,Rondônia,-,-,-,-,0.456321,0.449338,0.460385,0.517479,0.451794,0.427934,0.353360
2,Pessoas ocupadas (mil pessoas),Unidade da Federação,Rondônia,-,-,-,-,722.640940,749.836219,737.387514,745.379703,725.301308,748.225769,749.757706
3,CV - Pessoas ocupadas (%),Unidade da Federação,Rondônia,-,-,-,-,2.033890,2.117725,1.811941,1.780013,2.161756,1.828947,1.687723
4,Pessoas desocupadas (mil pessoas),Unidade da Federação,Rondônia,-,-,-,-,64.270072,74.731534,73.993264,74.143812,72.903672,71.454508,77.397970


In [ ]:
# Convert the Healthy/Saúde DataFrame to a Parquet file - Dados com detalhamento UF
df_saude.to_parquet("flat_file_saude_202011.parquet", engine="pyarrow", index=False)

print("DataFrame Saúde DataSet successfully converted to Parquet format!")

# Read the Saúde Parquet file
parquet_df_saude = pd.read_parquet("flat_file_saude_202011.parquet")

DataFrame Saúde DataSet successfully converted to Parquet format!


In [ ]:
# Convert the Healthy/Saúde DataFrame to a Parquet file - Dados com detalhamento UF
df_saude_br.to_parquet("flat_file_saude_br_202011.parquet", engine="pyarrow", index=False)

print("DataFrame Saúde DataSet successfully converted to Parquet format!")

# Read the Saúde Parquet file
parquet_df_saude_br = pd.read_parquet("flat_file_saude_br_202011.parquet")

DataFrame Saúde DataSet successfully converted to Parquet format!


In [ ]:
# Convert the Work/Trabalho DataFrame to a Parquet file
df_trabalho.to_parquet("flat_file_trabalho_202011.parquet", engine="pyarrow", index=False)

print("DataFrame Trabalho DataSet successfully converted to Parquet format!")

# Read the Trabalho Parquet file
parquet_df_trabalho = pd.read_parquet("flat_file_trabalho_202011.parquet")

DataFrame Trabalho DataSet successfully converted to Parquet format!


#Connect to Google Drive - BigQuery

In [ ]:
!pip install google-cloud-bigquery pandas pyarrow

In [ ]:
#Authenticate with your Google Cloud credentials:
from google.colab import auth
auth.authenticate_user()

print("Authenticated successfully!")

Authenticated successfully!


In [ ]:
from google.cloud import bigquery

#Connect to the Google Cloud Project ID 'tech-challenge-3-group-20'
project_id = "tech-challenge-3-group-20"
client = bigquery.Client(project=project_id)

In [ ]:
#Upload Parquet Files to Google Cloud Storage

from google.cloud import storage

# Initialize Cloud Storage client
storage_client = storage.Client()

# Create a bucket or use an existing one
bucket_name = "bucket-quickstart_tech-challenge-3-group-20"
bucket = storage_client.bucket(bucket_name)

# Upload Parquet files
parquet_files = ["flat_file_saude_202011.parquet", "flat_file_saude_br_202011.parquet", "flat_file_trabalho_202011.parquet"]
for file in parquet_files:
    blob = bucket.blob(file)
    blob.upload_from_filename(file)
    print(f"Uploaded {file} to GCS bucket {bucket_name}.")

Uploaded flat_file_saude_202011.parquet to GCS bucket bucket-quickstart_tech-challenge-3-group-20.
Uploaded flat_file_saude_br_202011.parquet to GCS bucket bucket-quickstart_tech-challenge-3-group-20.
Uploaded flat_file_trabalho_202011.parquet to GCS bucket bucket-quickstart_tech-challenge-3-group-20.


In [ ]:
#Upload colab notebook to Google BigQuery (Step no longer required)

# # Set GCS bucket and file details
# file_name = "Tech_Challenge_Fase3_import_files.ipynb"  # Replace with the name of your Colab notebook
# destination_blob_name = "path/in/gcs/Tech_Challenge_Fase3_import_files.ipynb.ipynb"

# # Upload notebook to GCS
# storage_client = storage.Client()
# bucket = storage_client.bucket(bucket_name)
# blob = bucket.blob(destination_blob_name)
# blob.upload_from_filename(file_name)

# print(f"Notebook {file_name} uploaded to {bucket_name}/{destination_blob_name}.")

In [ ]:
parquet_df_trabalho.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17602 entries, 0 to 17601
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Indicador                17602 non-null  object 
 1   Nível Territorial        17602 non-null  object 
 2   Abertura Territorial     17602 non-null  object 
 3   Variável de abertura 1   17602 non-null  object 
 4   Categoria de abertura 1  17602 non-null  object 
 5   Variável de abertura 2   17602 non-null  object 
 6   Categoria de abertura 2  17602 non-null  object 
 7   Maio                     17602 non-null  float64
 8   Junho                    17602 non-null  float64
 9   Julho                    17602 non-null  float64
 10  Agosto                   17602 non-null  float64
 11  Setembro                 17602 non-null  float64
 12  Outubro                  17602 non-null  float64
 13  Novembro                 17602 non-null  float64
dtypes: float64(7), object(

In [ ]:
parquet_df_saude.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17334 entries, 0 to 17333
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Indicador                17334 non-null  object 
 1   Nível Territorial        17334 non-null  object 
 2   Abertura Territorial     17334 non-null  object 
 3   Variável de abertura 1   17334 non-null  object 
 4   Categoria de abertura 1  17334 non-null  object 
 5   Variável de abertura 2   17334 non-null  object 
 6   Categoria de abertura 2  17334 non-null  object 
 7   Maio                     17334 non-null  float64
 8   Junho                    17334 non-null  float64
 9   Julho                    17334 non-null  float64
 10  Agosto                   17334 non-null  float64
 11  Setembro                 17334 non-null  float64
 12  Outubro                  17334 non-null  float64
 13  Novembro                 17334 non-null  float64
dtypes: float64(7), object(

In [ ]:
parquet_df_saude_br.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13030 entries, 0 to 13029
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Indicador                13030 non-null  object 
 1   Nível Territorial        13030 non-null  object 
 2   Abertura Territorial     13030 non-null  object 
 3   Variável de abertura 1   13030 non-null  object 
 4   Categoria de abertura 1  13030 non-null  object 
 5   Variável de abertura 2   13030 non-null  object 
 6   Categoria de abertura 2  13030 non-null  object 
 7   Maio                     13030 non-null  float64
 8   Junho                    13030 non-null  float64
 9   Julho                    13030 non-null  float64
 10  Agosto                   13030 non-null  float64
 11  Setembro                 13030 non-null  float64
 12  Outubro                  13030 non-null  float64
 13  Novembro                 13030 non-null  float64
dtypes: float64(7), object(

In [ ]:
parquet_df_saude

,Indicador,Nível Territorial,Abertura Territorial,Variável de abertura 1,Categoria de abertura 1,Variável de abertura 2,Categoria de abertura 2,Maio,Junho,Julho,Agosto,Setembro,Outubro,Novembro
0,Pessoas que apresentaram algum dos sintoma(s) ...,Unidade da Federação,Rondônia,-,-,-,-,181.943330,134.270923,91.120006,96.641585,77.548385,51.144917,41.357501
1,CV - Pessoas que apresentaram algum dos sintom...,Unidade da Federação,Rondônia,-,-,-,-,8.318182,7.469208,8.937854,10.752584,11.760041,14.545605,15.459794
2,Percentual de pessoas que apresentaram algum d...,Unidade da Federação,Rondônia,-,-,-,-,10.203347,7.523230,5.100973,5.405325,4.333607,2.855605,2.307125
3,CV - Percentual de pessoas que apresentaram al...,Unidade da Federação,Rondônia,-,-,-,-,8.318182,7.469208,8.937854,10.752584,11.760041,14.545605,15.459794
4,Pessoas que apresentaram sintomas referenciado...,Unidade da Federação,Rondônia,-,-,-,-,29.952155,27.137167,15.521570,24.030210,11.512330,9.253908,7.788923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17329,CV - Percentual de pessoas com diagnóstico méd...,Unidade da Federação,Distrito Federal,-,-,Tipo de comorbidade,Hipertensão,0.000000,0.000000,3.591887,3.875471,3.444609,3.557764,3.416248
17330,CV - Percentual de pessoas com diagnóstico méd...,Unidade da Federação,Distrito Federal,-,-,Tipo de comorbidade,Asma/Bronquite/Enfisema/Doença respiratória cr...,0.000000,0.000000,5.654702,6.392432,6.273995,7.105238,6.842539
17331,CV - Percentual de pessoas com diagnóstico méd...,Unidade da Federação,Distrito Federal,-,-,Tipo de comorbidade,Doença(s) do coração,0.000000,0.000000,8.192804,8.336468,8.492607,8.186854,8.171761
17332,CV - Percentual de pessoas com diagnóstico méd...,Unidade da Federação,Distrito Federal,-,-,Tipo de comorbidade,Depressão,0.000000,0.000000,8.590834,8.362686,8.249383,10.430050,8.995588


In [ ]:
# prompt: show distinct values of column indicador of df parquet_df_saude

print(parquet_df_saude['Indicador'].unique())

['Pessoas que apresentaram algum dos sintoma(s) (mil pessoas)'
 'CV - Pessoas que apresentaram algum dos sintoma(s) (%)'
 'Percentual de pessoas que apresentaram algum dos sintomas no total da população (%) '
 'CV - Percentual de pessoas que apresentaram algum dos sintomas no total da população (%) '
 'Pessoas que apresentaram sintomas referenciados conjugados3 (mil pessoas)'
 'CV - Pessoas que apresentaram sintomas referenciados conjugados3 (%) '
 'Percentual de pessoas que apresentaram sintomas referenciados conjugados3 no total da população (%)'
 'CV - Percentual de pessoas que apresentaram sintomas referenciados conjugados3 no total da população (%)'
 'Distribuição das pessoas que apresentaram algum dos sintoma(s) (%)'
 'CV - Distribuição das pessoas que apresentaram algum dos sintoma(s) (%)'
 'Distribuição das pessoas que apresentaram sintomas referenciados conjugados3 (%)'
 'CV - Distribuição das pessoas que apresentaram sintomas referenciados conjugados3 (%)'
 'População residen

In [ ]:
parquet_df_saude.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17334 entries, 0 to 17333
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Indicador                17334 non-null  object 
 1   Nível Territorial        17334 non-null  object 
 2   Abertura Territorial     17334 non-null  object 
 3   Variável de abertura 1   17334 non-null  object 
 4   Categoria de abertura 1  17334 non-null  object 
 5   Variável de abertura 2   17334 non-null  object 
 6   Categoria de abertura 2  17334 non-null  object 
 7   Maio                     17334 non-null  float64
 8   Junho                    17334 non-null  float64
 9   Julho                    17334 non-null  float64
 10  Agosto                   17334 non-null  float64
 11  Setembro                 17334 non-null  float64
 12  Outubro                  17334 non-null  float64
 13  Novembro                 17334 non-null  float64
dtypes: float64(7), object(

In [ ]:
df_saude.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17334 entries, 0 to 17333
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Indicador                17334 non-null  object 
 1   Nível Territorial        17334 non-null  object 
 2   Abertura Territorial     17334 non-null  object 
 3   Variável de abertura 1   17334 non-null  object 
 4   Categoria de abertura 1  17334 non-null  object 
 5   Variável de abertura 2   17334 non-null  object 
 6   Categoria de abertura 2  17334 non-null  object 
 7   Maio                     17334 non-null  float64
 8   Junho                    17334 non-null  float64
 9   Julho                    17334 non-null  float64
 10  Agosto                   17334 non-null  float64
 11  Setembro                 17334 non-null  float64
 12  Outubro                  17334 non-null  float64
 13  Novembro                 17334 non-null  float64
dtypes: float64(7), object(

In [ ]:
# prompt: show distinct values of column indicador of df parquet_df_saude

print(parquet_df_saude_br['Abertura Territorial'].unique())

['Brasil' 'Norte' 'Nordeste' 'Sudeste' 'Sul' 'Centro-Oeste']


In [ ]:
print(df_saude_br['Abertura Territorial'].unique())

['Brasil' 'Norte' 'Nordeste' 'Sudeste' 'Sul' 'Centro-Oeste']
